<a href="https://colab.research.google.com/github/yutan0565/colab_git/blob/main/code/Resnet50_%EC%98%88%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기본 모델 형성

In [1]:
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 237 kB 4.2 MB/s 


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import os
import datetime
import time
import tempfile
import pathlib

from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
import tensorflow_model_optimization as tfmot

In [3]:
# Cifar_10 데이터 불러오기
(raw_train_x, raw_train_y), (raw_test_x, raw_test_y) = tf.keras.datasets.cifar10.load_data()

# train, val, test 분리 및 scale 조정
train_x = raw_train_x[:45000].astype(np.float32)/255.0
valid_x = raw_train_x[45000:].astype(np.float32)/255.0
test_x = raw_test_x.astype(np.float32)/255.0


train_y = raw_train_y[:45000]
valid_y = raw_train_y[45000:]
test_y = raw_test_y
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

170508288/170498071 [==============================] - 11s 0us/step


In [4]:
# 모델 불러오기 Conv-layer 
resnet = ResNet50(  #weights = 'imagenet',  #그냥 초기화 하는거면, 이거 지우기
                            include_top = False,
                            input_shape=(32,32,3)
                            )

# FC layer는 다른거 사용   --  Class 10 개 분류
fc_layer = keras.Sequential([
                             layers.Flatten(),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(10, activation = "sigmoid")
                             ])

model = keras.Sequential([resnet,
                          fc_layer
                          ])
model.summary()


94781440/94765736 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 sequential (Sequential)     (None, 10)                4207626   
                                                                 
Total params: 27,795,338
Trainable params: 27,742,218
Non-trainable params: 53,120
_________________________________________________________________


In [5]:
# Callback 함수 지정 해주기      학습하는 동안 설정해줄것
early_stop = EarlyStopping(patience=30) 
mc = ModelCheckpoint("./best_model/resnet_original_checkpoint", 
                     save_best_only=True,
                     monitor = 'val_loss',
                     verbose = 1,
                     mode = 'min') 
reduce_lr  = ReduceLROnPlateau(monitor = 'val_loss',
                               factor=0.5, 
                               patience=5
                               ) 

#optimizer 조정 해주기
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [6]:
# optimizer, loss 함수를 정의하고,  학습 준비를 한다,  metrics 는 어떤 일이 발생하는지 보여줄 것들
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])



# 한번에 몇개의 데이터 학습하고 가중치 갱신할지 
model.fit(train_x, train_y,
          epochs=100,
          verbose=1,
          batch_size=32,
          #validation_split = 0.1
          validation_data = (valid_x, valid_y),
          callbacks = [early_stop, reduce_lr , mc]
          )

Epoch 1/100
1407/1407 [==============================] - ETA: 0s - loss: 1.1715 - accuracy: 0.5952
Epoch 1: val_loss improved from inf to 0.78046, saving model to ./best_model/resnet_original_checkpoint
INFO:tensorflow:Assets written to: ./best_model/resnet_original_checkpoint/assets
1407/1407 [==============================] - 220s 142ms/step - loss: 1.1715 - accuracy: 0.5952 - val_loss: 0.7805 - val_accuracy: 0.7352 - lr: 1.0000e-04
Epoch 2/100
1407/1407 [==============================] - ETA: 0s - loss: 0.7306 - accuracy: 0.7529
Epoch 2: val_loss improved from 0.78046 to 0.64495, saving model to ./best_model/resnet_original_checkpoint
INFO:tensorflow:Assets written to: ./best_model/resnet_original_checkpoint/assets
1407/1407 [==============================] - 175s 124ms/step - loss: 0.7306 - accuracy: 0.7529 - val_loss: 0.6450 - val_accuracy: 0.7808 - lr: 1.0000e-04
Epoch 3/100
1407/1407 [==============================] - ETA: 0s - loss: 0.5562 - accuracy: 0.8110
Epoch 3: val_loss i

In [7]:
loss, acc = model.evaluate(test_x, test_y)
print("loss=",loss)
print("acc=",acc)

# soft-max 형태로 결과가 나옴
y_ = model.predict(test_x)

# 모든 test_x 에 대해서 예측을 진행
predicted = np.argmax(y_, axis=1)
print(predicted)


313/313 [==============================] - 9s 28ms/step - loss: 1.3100 - accuracy: 0.8398
loss= 1.3100165128707886
acc= 0.8398000001907349
[3 1 8 ... 5 0 7]


In [8]:
# Check-Point 형태로 가장 마지막 모델 저장
# model.save("./saved_model/resnet_original.h5")
# model.save("./saved_model/resnet_original_checkpoint")

## int_8 Quantization 진행

In [9]:
# 구조까지 들어가 있는거
model = tf.keras.models.load_model('./best_model/resnet_original_checkpoint')

In [10]:
# Quantization을 위한 point 설정
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_x).batch(1).take(100):
    yield [input_value]

In [11]:
# 그냥 파일 형태만 tflite로 변환 (float 형태임)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpqes2b8jb/assets


In [12]:
# int 8 로 quantization 진행하기
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpswecn3iw/assets


INFO:tensorflow:Assets written to: /tmp/tmpswecn3iw/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [13]:
# Quantization 모델의 input, output 변환 잘 되었는지 확인 하기
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [14]:
# 모델 저장 해주는 과정
# tflite 형태 모델을 어디에 저장 해줄까, 폴더 있으면 패스, 없으면 만들어 주기
tflite_models_dir = pathlib.Path("./tflite/resnet_tflite/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)


In [15]:
# 일반 모델 tflite 형태로 저장
tflite_model_file = tflite_models_dir/"resnet_original_tflite.tflite"
open(tflite_model_file, "wb") .write(tflite_model)

# Quantized 모델 tflite 형태로 저장
tflite_model_quant_file = tflite_models_dir/"resnet_quantization_tflite.tflite"
open(tflite_model_quant_file, "wb") .write(tflite_model_quant)

#tflite_model_quant_file.write_bytes(tflite_model_quant)

28470720

In [16]:
# tflite 모델 평가에서 사용되는 실행 함수
def run_tflite_model(tflite_file, test_image_indices):
  global test_x

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_x[test_image_index]
    test_label = test_y[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

  return predictions

In [17]:
def evaluate_model(tflite_file, model_type):
  global test_x
  global test_y

  test_image_indices = range(test_x.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_y.reshape(-1)== predictions) * 100) / len(test_x)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_x)))

In [ ]:
# 평가는 뒤에 몰아주기
# evaluate_model(tflite_model_file, model_type="Float")
# evaluate_model(tflite_model_quant_file, model_type="Quantized")

In [18]:
# FPS 측정용 함수
def run_tflite_time(tflite_file, test_image_indices):
  global test_x

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  fps = np.zeros((len(test_image_indices),))
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_x[test_image_index]
    test_label = test_y[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

        
    start = time.time()
    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    end = time.time()
    fps[i] = (end - start)

  return fps

In [19]:
# Change this to test a different image
def test_model_time(tflite_file, model_type):
  global test_x
  test_image_indices = range(test_x.shape[0])
  fps = run_tflite_time(tflite_file, test_image_indices)
  print(np.mean(fps))
  print(1 / np.mean(fps))
  print('%s model FPS is %.4f%% (Number of test samples=%d)' % (model_type, 1 / np.mean(fps), len(test_x)))

In [20]:
# FPS 측정 뒤로 미루기
# test_model_time(tflite_model_file, model_type="Float")
# test_model_time(tflite_model_quant_file,  model_type="Quantized")

# Pruning 진행

In [21]:
model = tf.keras.models.load_model('./best_model/resnet_original_checkpoint')
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

# Pruning을 위한 변수 설정
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 32
epochs = 30
validation_split = 0.1 

num_images = train_x.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

In [ ]:
# Sparsity = 0.2,   pruning하고 다시 학습 시키기
pruning_params_2 = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.2,
                                                               begin_step=0,
                                                               end_step=-1,
                                                               frequency = 100
                                                               )
}

resnet_pruning_pruning_2 = prune_low_magnitude(model, **pruning_params_2)

resnet_pruning_pruning_2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

resnet_pruning_pruning_2.fit(train_x, train_y,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)


Epoch 1/30


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


   6/1266 [..............................] - ETA: 4:12 - loss: 1.5084 - accuracy: 0.6719WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1233s vs `on_train_batch_end` time: 0.2779s). Check your callbacks.


1266/1266 [==============================] - 281s 186ms/step - loss: 1.3685 - accuracy: 0.5514 - val_loss: 1.6398 - val_accuracy: 0.5196
Epoch 2/30
1266/1266 [==============================] - 231s 182ms/step - loss: 1.1698 - accuracy: 0.6068 - val_loss: 1.2508 - val_accuracy: 0.5798
Epoch 3/30
1266/1266 [==============================] - 227s 179ms/step - loss: 1.0548 - accuracy: 0.6467 - val_loss: 1.4482 - val_accuracy: 0.5127
Epoch 4/30
1266/1266 [==============================] - 229s 181ms/step - loss: 0.9992 - accuracy: 0.6648 - val_loss: 1.4227 - val_accuracy: 0.5462
Epoch 5/30
1266/1266 [==============================] - 225s 178ms/step - loss: 0.9339 - accuracy: 0.6833 - val_loss: 1.0791 - val_accuracy: 0.6438
Epoch 6/30
1266/1266 [==============================] - 226s 178ms/step - loss: 1.0186 - accuracy: 0.6551 - val_loss: 1.0622 - val_accuracy: 0.6380
Epoch 7/30
1266/1266 [==============================] - 228s 180ms/step - loss: 1.1023 - accuracy: 0.6264 - val_loss: 1.038

In [ ]:
# Sparsity = 0.4,   pruning하고 다시 학습 시키기
pruning_params_4 = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.4,
                                                               begin_step=0,
                                                               end_step=-1,
                                                               frequency = 100
                                                               )
}

resnet_pruning_pruning_4 = prune_low_magnitude(model, **pruning_params_4)

resnet_pruning_pruning_4.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

resnet_pruning_pruning_4.fit(train_x, train_y,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

In [ ]:
# Sparsity = 0.6,   pruning하고 다시 학습 시키기
pruning_params_6 = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.6,
                                                               begin_step=0,
                                                               end_step=-1,
                                                               frequency = 100
                                                               )
}

resnet_pruning_pruning_6 = prune_low_magnitude(model, **pruning_params_6)

resnet_pruning_pruning_6.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

resnet_pruning_pruning_6.fit(train_x, train_y,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

In [ ]:
# Sparsity = 0.8,   pruning하고 다시 학습 시키기
pruning_params_8 = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(target_sparsity=0.8,
                                                               begin_step=0,
                                                               end_step=-1,
                                                               frequency = 100
                                                               )
}

resnet_pruning_pruning_8 = prune_low_magnitude(model, **pruning_params_8)

resnet_pruning_pruning_8.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

resnet_pruning_pruning_8.fit(train_x, train_y,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

In [ ]:
# _, model_for_pruning_accuracy = resnet_pruning_pruning_2.evaluate(
#    test_x, test_y, verbose=0)

# print('Baseline test accuracy:', baseline_model_accuracy) 
# print('Pruned test accuracy:', model_for_pruning_accuracy)

In [ ]:
resnet_pruning_pruning_2_eport = tfmot.sparsity.keras.strip_pruning(resnet_pruning_pruning_2)
resnet_pruning_pruning_4_eport = tfmot.sparsity.keras.strip_pruning(resnet_pruning_pruning_4)
resnet_pruning_pruning_6_eport = tfmot.sparsity.keras.strip_pruning(resnet_pruning_pruning_6)
resnet_pruning_pruning_8_eport = tfmot.sparsity.keras.strip_pruning(resnet_pruning_pruning_8)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(resnet_pruning_pruning_2_eport)
resnet_pruning_pruning_2_tflite = converter.convert()
converter = tf.lite.TFLiteConverter.from_keras_model(resnet_pruning_pruning_4_eport)
resnet_pruning_pruning_4_tflite = converter.convert()
converter = tf.lite.TFLiteConverter.from_keras_model(resnet_pruning_pruning_6_eport)
resnet_pruning_pruning_6_tflite = converter.convert()
converter = tf.lite.TFLiteConverter.from_keras_model(resnet_pruning_pruning_8_eport)
resnet_pruning_pruning_8_tflite = converter.convert()

In [ ]:
resnet_pruning_pruning_2_file = tflite_models_dir/"resnet_pruning_pruning_2_tflite.tflite"
open(resnet_pruning_pruning_2_file, "wb") .write(resnet_pruning_pruning_2_tflite)

resnet_pruning_pruning_4_file = tflite_models_dir/"resnet_pruning_pruning_4_tflite.tflite"
open(resnet_pruning_pruning_4_file, "wb") .write(resnet_pruning_pruning_4_tflite)

resnet_pruning_pruning_6_file = tflite_models_dir/"resnet_pruning_pruning_6_tflite.tflite"
open(resnet_pruning_pruning_6_file, "wb") .write(resnet_pruning_pruning_6_tflite)

resnet_pruning_pruning_8_file = tflite_models_dir/"resnet_pruning_pruning_8_tflite.tflite"
open(resnet_pruning_pruning_8_file, "wb") .write(resnet_pruning_pruning_8_tflite)

In [ ]:
# 모델의 bytes 크기 
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  print(os.path.getsize(zipped_file))

## 결과창

In [ ]:
evaluate_model(tflite_model_file, model_type="Float")
evaluate_model(tflite_model_quant_file, model_type="Quantized")
evaluate_model(resnet_pruning_pruning_2_file, model_type="Pruning_2")
evaluate_model(resnet_pruning_pruning_4_file, model_type="Pruning_4")
evaluate_model(resnet_pruning_pruning_6_file, model_type="Pruning_6")
evaluate_model(resnet_pruning_pruning_8_file, model_type="Pruning_8")

In [ ]:
test_model_time(tflite_model_file, model_type="Float")
test_model_time(tflite_model_quant_file,  model_type="Quantized")

test_model_time(resnet_pruning_pruning_2_file,  model_type="Pruning_2")
test_model_time(resnet_pruning_pruning_4_file,  model_type="Pruning_4")
test_model_time(resnet_pruning_pruning_6_file,  model_type="Pruning_6")
test_model_time(resnet_pruning_pruning_8_file,  model_type="Pruning_8")

In [ ]:
get_gzipped_model_size(tflite_model_file)
get_gzipped_model_size(tflite_model_quant_file)

get_gzipped_model_size(resnet_pruning_pruning_2_file)
get_gzipped_model_size(resnet_pruning_pruning_4_file)
get_gzipped_model_size(resnet_pruning_pruning_6_file)
get_gzipped_model_size(resnet_pruning_pruning_8_file)